# Effect of synchronization noise on the classic ML estimator

This notebook reproduces the toy examples of **Figure 1b** which characterizes the effect of synchronization noise on the classic ML estimator.

---

Loads libraries.

In [ ]:
# External libs
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx

# Use `tick` to simulate synthetic data and for the baseline `Classic MLE` estimator
from tick.hawkes.simulation import SimuHawkesExpKernels
from tick.hawkes.inference import HawkesADM4

%matplotlib inline
plt.rcParams['font.size'] = 16

# Graph drawing parameters
nx_draw_params = {
    'pos':{0: (0,0), 1: (1, 0)}, 
    'node_size': 2e3, 'node_color': 'w', 
    'labels': {0: '$N_A$', 1: '$N_B$'},
    'edgecolors':'k', 'arrowsize': 50, 'width': 2,
    'font_size': 20
}

Initialize random seed.

In [ ]:
np.random.seed(12345678)

---

## 1. Define parameters of the simulations

In [ ]:
n_nodes = 2         # Number of nodes
decay = 1.0         # Expoenential decay
end_time = 10e3     # Length of the observation window

# Background intensity
mu = 0.05
baseline = mu * np.ones(n_nodes, dtype='float')

# Excitation matrix
adjacency = np.array([[0.95, 0.0],
                      [0.95, 0.0]])

In [ ]:
print('True baseline:')
print(baseline.round(2))
print('True adjacency:')
print(adjacency.round(2))

# Draw the graph
print('\nEstimated excitation graph:')
graph_true = nx.DiGraph(adjacency.T)
plt.figure(figsize=(8, 1))
nx.draw_networkx(graph_true, **nx_draw_params)
plt.axis('off');

---

## 2. Run the simulations

Build an utility function to generate noisy sample from an MHP

In [ ]:
def generate_data(adjacency, decay, baseline, end_time, z_true):
    """
    Generate a realization of a MHP with synchronization noise
    """
    # Compute the observation window adjusted to the noise (to avoid edge effects)
    adjusted_end_time = end_time + z_true.min() - z_true.max()
    adjusted_start_time = z_true.max()
    # Simulate a realization of the process
    simu = SimuHawkesExpKernels(adjacency=adjacency,
                                decays=decay,
                                baseline=baseline,
                                end_time=adjusted_end_time,
                                verbose=False)
    simu.simulate()
    # Add synchronization noise to it
    events = []
    for m, orig_events_m in enumerate(simu.timestamps):
        events_m = orig_events_m + z_true[m] - adjusted_start_time
        events_m = events_m[(events_m >= 0) & (events_m < end_time)]
        events.append(events_m)
    return events

Run the experiments as follows:
- First fix $z^A$ to an arbitrary value, e.g. $z^A = 0$.
- Then iterate over a range of `N` values of $z^B$ between $-10$ and $10$.
- Average the results of each value of $[z^A, z^B]$ over `K` simulations.

**WARNING:** This cell is quite computationally expensive and may take some time to run.

In [ ]:
N = 101  # Number of noise values to choose for `z^B`
K = 10   # Number of simulations per value of `z^B`

# Set the synchronization noise values in each dimension
z_1 = 0.0 # Arbitrarily fix z_1 to zero
z_2_range = np.linspace(-10.0, 10.0, N) # Vary `z_2` from -10 to 10

# Init the results of the experiments in this array
adjacency_arr = np.zeros((N, K, 2, 2))

# Iterate of values of `z^B`
for i, z_2 in enumerate(z_2_range):
    # Set the value of the noise
    z = np.hstack((z_1, z_2))
    # Run `K` experiments for this value of `z`
    for k in range(K):
        # Generate a noisy realization of the MHP
        events = generate_data(adjacency=adjacency, decay=decay, baseline=baseline, end_time=end_time, z_true=z)
        # Apply the classic MLE
        learner = HawkesADM4(decay, C=1e3, lasso_nuclear_ratio=1.0, n_threads=2)
        learner.fit(events, end_time)
        adjacency_arr[i,k] = learner.adjacency
        print(f'{i*K + k + 1:d}/{N*K:d} experiments performed...', end='\r', flush=True)
print()

---

## 3. Reproduce Figure 1b from the paper

Plot the results.

In [ ]:
plt.figure(figsize=(12, 4))
plt.grid()
plt.errorbar(z_2_range, np.mean(adjacency_arr[:,:,0,1], axis=1), np.std(adjacency_arr[:,:,0,1], axis=1), 
             c='C0', label=r'Estimated $\hat{\alpha}_{AB}(z^B - z^A)$')
plt.errorbar(z_2_range, np.mean(adjacency_arr[:,:,1,0], axis=1), np.std(adjacency_arr[:,:,1,0], axis=1), 
             c='C1', label=r'Estimated $\hat{\alpha}_{BA}(z^B - z^A)$')
plt.axhline(y=adjacency[0,1], c='C0', ls='--', label=r'Ground truth $\alpha_{AB}$')
plt.axhline(y=adjacency[1,0], c='C1', ls='--', label=r'Ground truth $\alpha_{BA}$')
plt.xlabel(r'$z^B - z^A$')
plt.ylabel('Kernel coefficients')
plt.legend()
plt.tight_layout();